**Capstone Project - The Battle of Neighborhoods (Week 1-2)**
Business Problem section
Background
According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.

**Business Problem**
In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

**Data section**
Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

 **Methodology section**
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

1. **Collect Inspection Data**

After importing the necessary libraries, we download the data from the HM Land Registry website as follows:

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

/bin/bash: conda: command not found
/bin/bash: conda: command not found
Libraries imported.


In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

Before using data, we will have to explore and understand it.

2. Explore and Understand Data
We read the dataset that we collected from the HM Land Registry website into a pandas' data frame and display the first five rows of it as follows:

In [3]:
df_ppd.head(5)

,{726BF13A-993E-0A46-E053-6C04A8C01D0D},75000,2018-06-20 00:00,DL13 4BA,T,N,F,21,Unnamed: 8,HIGHFIELDS,TOW LAW,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM.1,A,A.1
0,{726BF13A-993F-0A46-E053-6C04A8C01D0D},115000,2018-07-13 00:00,DL17 9LB,S,N,F,4,NaN,THE LANE,WEST CORNFORTH,FERRYHILL,COUNTY DURHAM,COUNTY DURHAM,A,A
1,{726BF13A-9940-0A46-E053-6C04A8C01D0D},24000,2018-04-10 00:00,SR7 9AG,F,N,L,20B,NaN,WOODS TERRACE,MURTON,SEAHAM,COUNTY DURHAM,COUNTY DURHAM,A,A
2,{726BF13A-9941-0A46-E053-6C04A8C01D0D},56000,2018-06-22 00:00,DL5 5PS,T,N,F,6,NaN,HEILD CLOSE,NaN,NEWTON AYCLIFFE,COUNTY DURHAM,COUNTY DURHAM,A,A
3,{726BF13A-9942-0A46-E053-6C04A8C01D0D},220000,2018-05-25 00:00,DL16 7HE,D,N,F,25,NaN,BECKWITH CLOSE,KIRK MERRINGTON,SPENNYMOOR,COUNTY DURHAM,COUNTY DURHAM,A,A
4,{726BF13A-9943-0A46-E053-6C04A8C01D0D},58000,2018-05-09 00:00,DL14 6FH,F,N,L,23,NaN,AINTREE DRIVE,NaN,BISHOP AUCKLAND,COUNTY DURHAM,COUNTY DURHAM,A,A


In [4]:
df_ppd.shape

(1032408, 16)

Our dataset consists of over 1030000 rows and 16 columns. We will now prepare and preprocess data accordingly.

3. **Data preparation and preprocessing**
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

Rename the column names
Format the date column
Sort data by date of sale
Select data only for the city of London
Make a list of street names in London
Calculate the street-wise average price of the property
Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
Join the data to find the coordinates of locations which fit into client's budget
Plot recommended locations on London map along with current market prices




In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [6]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [7]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [9]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [10]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [12]:
# Display the dataframe
df_affordable

,Street,Avg_Price
196,ALBION SQUARE,2450000.0
390,ANHALT ROAD,2435000.0
405,ANSDELL TERRACE,2250000.0
422,APPLEGARTH ROAD,2400000.0
857,BARONSMEAD ROAD,2375000.0
...,...,...
13736,WILFRED STREET,2410538.5
13762,WILLOW BRIDGE ROAD,2425000.0
13782,WILSON STREET,2257500.0
13811,WINCHENDON ROAD,2350000.0


In [13]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [14]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 196
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 196, dtype: object
item.Street only: ALBION SQUARE
index: 390
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 390, dtype: object
item.Street only: ANHALT ROAD
index: 405
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 405, dtype: object
item.Street only: ANSDELL TERRACE
index: 422
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 422, dtype: object
item.Street only: APPLEGARTH ROAD
index: 857
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 857, dtype: object
item.Street only: BARONSMEAD ROAD
index: 983
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 983, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1105
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name: 1105, dtype: object
item.Street only: BELVEDERE DRIVE
index: 1218
item: Street       BICKENHALL STREET
Avg_Price           2.2085e+06
N

In [15]:
geolocator = Nominatim()

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


In [16]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
df_affordable

,Street,Avg_Price,city_coord
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)"
390,ANHALT ROAD,2435000.0,"(29.805547, -98.475534)"
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)"
422,APPLEGARTH ROAD,2400000.0,"(40.271236, -74.4711104)"
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)"
...,...,...,...
13736,WILFRED STREET,2410538.5,"(46.4369189, -63.8655639)"
13762,WILLOW BRIDGE ROAD,2425000.0,"(51.5431088, -0.0955079)"
13782,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)"
13811,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)"


In [18]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [19]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
196,ALBION SQUARE,2450000.0,"(-41.27375755, 173.28939323910353)",-41.273758,173.289393
390,ANHALT ROAD,2435000.0,"(29.805547, -98.475534)",29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,"(51.5000051, -0.1891537)",51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,"(40.271236, -74.4711104)",40.271236,-74.471110
857,BARONSMEAD ROAD,2375000.0,"(51.4773147, -0.239457)",51.477315,-0.239457
...,...,...,...,...,...
13736,WILFRED STREET,2410538.5,"(46.4369189, -63.8655639)",46.436919,-63.865564
13762,WILLOW BRIDGE ROAD,2425000.0,"(51.5431088, -0.0955079)",51.543109,-0.095508
13782,WILSON STREET,2257500.0,"(45.7004857, -121.52187527526078)",45.700486,-121.521875
13811,WINCHENDON ROAD,2350000.0,"(42.645331, -71.954714)",42.645331,-71.954714


In [20]:
df = df_affordable.drop(columns=['city_coord'])

In [21]:
df

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,40.271236,-74.471110
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457
...,...,...,...,...
13736,WILFRED STREET,2410538.5,46.436919,-63.865564
13762,WILLOW BRIDGE ROAD,2425000.0,51.543109,-0.095508
13782,WILSON STREET,2257500.0,45.700486,-121.521875
13811,WINCHENDON ROAD,2350000.0,42.645331,-71.954714


In [22]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/usr/local/lib/python3.7/dist-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [23]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [24]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3' # Foursquare ID
CLIENT_SECRET = 'QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD' # Foursquare Secret
VERSION = '20181206' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KI3TR0QO4JOKMFELOMF3WSOOI3HFNBF5YLW354MYWBKDHEX3
CLIENT_SECRET:QF4ZBLJRBV4BQX52DVWUPEHJ14A2UJABPCZARZQZYTKIISUD


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

4.**Modeling**

After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURY WALK
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CHEYNE ROW
CHISWICK MALL
CITY ROAD
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
COLLEGE CRESCENT
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRESCENT GROVE
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FLORAL STREET
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HARRIS STREET
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHWOOD HILL
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HORTENSIA ROAD
HOXTON SQUARE

In [28]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
4,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
...,...,...,...,...,...,...,...
4984,WILSON STREET,45.700486,-121.521875,Lake Taco,45.703056,-121.524445,Mexican Restaurant
4985,WILSON STREET,45.700486,-121.521875,The Shed,45.700849,-121.523560,Bar
4986,WILSON STREET,45.700486,-121.521875,Juanita's Marketa,45.700125,-121.524078,Grocery Store
4987,WILSON STREET,45.700486,-121.521875,Shell,45.696831,-121.523032,Gas Station


In [29]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,26,26,26,26,26,26
ANSDELL TERRACE,66,66,66,66,66,66
APPLEGARTH ROAD,5,5,5,5,5,5
BARONSMEAD ROAD,16,16,16,16,16,16
BEAUCLERC ROAD,2,2,2,2,2,2
...,...,...,...,...,...,...
WESTMORELAND PLACE,15,15,15,15,15,15
WHITFIELD STREET,100,100,100,100,100,100
WILFRED STREET,2,2,2,2,2,2


In [30]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 348 uniques categories.


In [31]:
location_venues.shape

(4989, 7)

In [32]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()


,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bathing Area,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,...,Street Art,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toll Booth,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Trattoria/Osteria,Tree,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Service,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Basketball Court,Bathing Area,Bay,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Boarding House,Boat or Ferry,Bookstore,...,Street Art,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vintage Store,Tiki Bar,Toll Booth,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Track Stadium,Trail,Train Station,Tram Station,Trattoria/Osteria,Tree,Turkish Restaurant,Vape Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,ALBION SQUARE,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.038462,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.076923,0.0,0.0,0.0,0.0,0.00,0.000000,0.038462,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.038462,...,0.0,0.0,0.0,0.038462,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,ANSDELL TERRACE,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.015152,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.030303,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.015152,0.0,0.0,0.015152,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.015152,0.0,0.0,0.0,0.0,0.0
2,APPLEGARTH ROAD,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,BARONSMEAD ROAD,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.062500,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,BEAUCLERC ROAD,0.000000,0.0,0.0,0.0,0.00,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,WESTMORELAND PLACE,0.000000,0.0,0.0,0.0,0.00,0.0,0.066667,0.0,0.0,0.0,0.066667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.066667,0.066667,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.066667,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0

In [34]:
london_grouped.shape

(144, 349)

In [35]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.19
1         Restaurant  0.08
2                Bar  0.08
3  Indian Restaurant  0.08
4                Pub  0.08


----ANSDELL TERRACE----
                venue  freq
0               Hotel  0.08
1          Restaurant  0.06
2   Indian Restaurant  0.06
3           Juice Bar  0.06
4  Italian Restaurant  0.05


----APPLEGARTH ROAD----
                  venue  freq
0     Convenience Store   0.4
1      Storage Facility   0.2
2  Fast Food Restaurant   0.2
3            Donut Shop   0.2
4                   ATM   0.0


----BARONSMEAD ROAD----
               venue  freq
0  Food & Drink Shop  0.12
1                Pub  0.06
2  Convenience Store  0.06
3    Thai Restaurant  0.06
4               Park  0.06


----BEAUCLERC ROAD----
                   venue  freq
0             Food Truck   0.5
1              Locksmith   0.5
2                    ATM   0.0
3  Outdoors & Recreation   0.0
4              Pawn Shop   0.0


----BELVEDERE D

In [36]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [38]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [39]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Bar,Indian Restaurant,Pub,Restaurant,Burger Joint,Fish & Chips Shop,Seafood Restaurant,Park,Supermarket
1,ANSDELL TERRACE,Hotel,Indian Restaurant,Juice Bar,Restaurant,Pub,Italian Restaurant,Clothing Store,English Restaurant,Sporting Goods Shop,Café
2,APPLEGARTH ROAD,Convenience Store,Storage Facility,Donut Shop,Fast Food Restaurant,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm
3,BARONSMEAD ROAD,Food & Drink Shop,Indie Movie Theater,Restaurant,Breakfast Spot,Movie Theater,Café,Farmers Market,Thai Restaurant,Park,Coffee Shop
4,BEAUCLERC ROAD,Food Truck,Locksmith,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant


In [40]:
venues_sorted.shape

(144, 11)

In [41]:
london_grouped.shape

(144, 349)

In [43]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [44]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([1, 4, 2, 4, 0, 1, 0, 2, 2, 1, 4, 4, 1, 0, 0, 1, 4, 2, 1, 3, 3, 4,
       1, 1, 2, 4, 3, 1, 2, 4, 0, 4, 0, 0, 3, 4, 4, 0, 2, 1, 0, 3, 2, 3,
       2, 2, 3, 2, 2, 4], dtype=int32)

In [45]:
#Dataframe to include Clusters

london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
196,ALBION SQUARE,2450000.0,-41.273758,173.289393
390,ANHALT ROAD,2435000.0,29.805547,-98.475534
405,ANSDELL TERRACE,2250000.0,51.500005,-0.189154
422,APPLEGARTH ROAD,2400000.0,40.271236,-74.471110
857,BARONSMEAD ROAD,2375000.0,51.477315,-0.239457


In [46]:
london_grouped_clustering.shape

(159, 4)

In [47]:
df.shape

(159, 4)

In [48]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [49]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [50]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,1,Café,Bar,Indian Restaurant,Pub,Restaurant,Burger Joint,Fish & Chips Shop,Seafood Restaurant,Park,Supermarket
390,ANHALT ROAD,2.435000e+06,29.805547,-98.475534,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
405,ANSDELL TERRACE,2.250000e+06,51.500005,-0.189154,2,Hotel,Indian Restaurant,Juice Bar,Restaurant,Pub,Italian Restaurant,Clothing Store,English Restaurant,Sporting Goods Shop,Café
422,APPLEGARTH ROAD,2.400000e+06,40.271236,-74.471110,4,Convenience Store,Storage Facility,Donut Shop,Fast Food Restaurant,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm
857,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,0,Food & Drink Shop,Indie Movie Theater,Restaurant,Breakfast Spot,Movie Theater,Café,Farmers Market,Thai Restaurant,Park,Coffee Shop
983,BEAUCLERC ROAD,2.480000e+06,30.207887,-81.628209,1,Food Truck,Locksmith,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1105,BELVEDERE DRIVE,2.340000e+06,-29.580059,31.088130,0,Convenience Store,Zoo Exhibit,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1218,BICKENHALL STREET,2.208500e+06,51.521201,-0.158908,2,Restaurant,Hotel,Sandwich Place,Greek Restaurant,Bar,Bakery,Breakfast Spot,Movie Theater,Café,Museum
1256,BIRCHLANDS AVENUE,2.217000e+06,51.448414,-0.160457,2,Pub,Brewery,Pizza Place,Gym / Fitness Center,Bakery,Coffee Shop,Lake,Chinese Restaurant,French Restaurant,Flower Shop
1556,BRAMPTON GROVE,2.456875e+06,51.570365,-0.283394,1,Middle Eastern Restaurant,Lake,Zoo Exhibit,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant


In [51]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
857,2375000.0,Food & Drink Shop,Indie Movie Theater,Restaurant,Breakfast Spot,Movie Theater,Café,Farmers Market,Thai Restaurant,Park,Coffee Shop
1105,2340000.0,Convenience Store,Zoo Exhibit,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
2071,2375000.0,Pub,Hotel,Indian Restaurant,Grocery Store,Yoga Studio,Park,Caribbean Restaurant,Gastropub,Café,Coffee Shop
2132,2379652.7,Pub,Grocery Store,Park,Hotel,Indian Restaurant,Yoga Studio,Coffee Shop,Sandwich Place,English Restaurant,Fish & Chips Shop
2946,2367500.0,Hotel,Pub,Garden,Coffee Shop,Café,Italian Restaurant,Mediterranean Restaurant,Chinese Restaurant,Bar,Gym / Fitness Center


In [53]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
196,2450000.0,Café,Bar,Indian Restaurant,Pub,Restaurant,Burger Joint,Fish & Chips Shop,Seafood Restaurant,Park,Supermarket
983,2480000.0,Food Truck,Locksmith,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
1556,2456875.0,Middle Eastern Restaurant,Lake,Zoo Exhibit,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1983,2492500.0,English Restaurant,Supermarket,Pub,Café,Dry Cleaner,Hardware Store,Fast Food Restaurant,Coffee Shop,Discount Store,Park
2139,2461000.0,Bakery,Furniture / Home Store,Fast Food Restaurant,Coffee Shop,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market


In [54]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
405,2250000.0,Hotel,Indian Restaurant,Juice Bar,Restaurant,Pub,Italian Restaurant,Clothing Store,English Restaurant,Sporting Goods Shop,Café
1218,2208500.0,Restaurant,Hotel,Sandwich Place,Greek Restaurant,Bar,Bakery,Breakfast Spot,Movie Theater,Café,Museum
1256,2217000.0,Pub,Brewery,Pizza Place,Gym / Fitness Center,Bakery,Coffee Shop,Lake,Chinese Restaurant,French Restaurant,Flower Shop
2228,2200000.0,Campground,Bed & Breakfast,Café,Zoo Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio
2640,2250000.0,Pub,Café,Garden,Plaza,Grocery Store,Park,Coffee Shop,French Restaurant,English Restaurant,Gift Shop


In [55]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2245,2.300000e+06,Farm,Zoo Exhibit,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Film Studio
2408,2.286679e+06,Pub,Italian Restaurant,Café,Coffee Shop,Bar,Park,French Restaurant,Portuguese Restaurant,Music Venue,Beer Store
2688,2.287500e+06,Pub,Reservoir,Gift Shop,Harbor / Marina,Art Museum,Brewery,Dance Studio,Cycle Studio,Factory,Falafel Restaurant
3379,2.298000e+06,Hotel,Zoo Exhibit,Fish & Chips Shop,Exhibit,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
4287,2.265000e+06,Coffee Shop,Café,Pub,Zoo Exhibit,Music Venue,Pizza Place,Market,Yoga Studio,Italian Restaurant,French Restaurant


In [57]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
390,2435000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
422,2400000.0,Convenience Store,Storage Facility,Donut Shop,Fast Food Restaurant,Zoo Exhibit,Film Studio,Exhibit,Factory,Falafel Restaurant,Farm
1635,2397132.0,Breakfast Spot,Grocery Store,Park,Coffee Shop,Gym / Fitness Center,Filipino Restaurant,Exhibit,Factory,Falafel Restaurant,Farm
1800,2400000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2161,2425000.0,Hotel,Italian Restaurant,Pub,Mediterranean Restaurant,Indian Restaurant,Restaurant,Chinese Restaurant,Garden,French Restaurant,Coffee Shop


**Results and Discussion section**

First of all, even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs.

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

**Conclusion**

To sum up, according to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we were posing was: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.